### Sprawozdanie - Singular Value Decomposition
<div style="text-align: right"> Wojciech Kosztyła </div>

#### Zadanie 1 - Wyszukiwarka

1. Przygotuj duży (> 1000 elementów) zbiór dokumentów tekstowych w języku angielskim (np. wybrany korpus tekstów, podzbiór artykułów Wikipedii, zbiór dokumentów HTML uzyskanych za pomocą Web crawlera, zbiór rozdziałów wyciętych z różnych książek)

2. Określ słownik słów kluczowych (termów) potrzebny do wyznaczenia wektorów cech bag-of-words (indeksacja). Przykładowo zbiorem takim może być unia wszystkich słów występujących we wszystkich tekstach.

3. Dla każdego dokumentu j wyznacz wektor cech bag-of-words dj zawierający częstości występowania poszczególnych słów (termów) w tekście.

4. Zbuduj rzadką macierz wektorów cech term-by-document matrix w której wektory cech ułożone są kolumnowo $A_{m×n} = [d1|d2|...|dn]$ (m jest liczbą termów w słowniku, a n liczbą dokumentów)

5. Przetwórz wstępnie otrzymany zbiór danych mnożąc elementy bag-of-words przez inverse document frequency. Operacja ta pozwoli na redukcję znaczenia często występujących słów.

$IDF(w) = log\frac{N}{n_{w}}$

gdzie $n_{w}$ jest liczbą dokumentów, w których występuje słowo w, a $N$ jest całkowitą liczbą dokumentów.

6. Napisz program pozwalający na wprowadzenie zapytania (w postaci sekwencji słów) przekształcanego następnie do reprezentacji wektorowej q (bag-of-words). Program ma zwrócić k dokumentów najbardziej zbliżonych do podanego zapytania q. Użyj korelacji między wektorami jako miary podobieństwa

$ cos(θ)_{j} = \frac{q^{T}d_{j}}{||q||*||d_{j}||}=\frac{q^{T}Ae_{j}}{||q||*||Ae_{j}||}$

7. Zastosuj normalizację wektorów cech dj i wektora q, tak aby miały one długość 1. Użyj zmodyﬁkowanej miary podobieństwa otrzymując

$|q^{T}A|= [|cos(θ)_{1}|,|cos(θ)_{2}|,...,|cos(θ)_{n}|]$

8. W celu usunięcia szumu z macierzy A zastosuj SVD i low rank approximation otrzymując

![text](Adnotacja 2020-05-30 140957.png)

oraz nową miarę podobieństwa

$cos(\phi)_{j} == \frac{q^{T}A_{k}e_{j}}{||q||*||A_{k}e_{j}||}$

9. Porównaj działanie programu bez usuwania szumu i z usuwaniem szumu. Dla jakiej wartości k wyniki wyszukiwania są najlepsze (subiektywnie). Zbadaj wpływ przekształcenia IDF na wyniki wyszukiwania.

